In [1]:
from compas_notebook.viewer import Viewer
from compas.geometry import Point, Line, Polyline, Vector
from compas.colors import Color
from compas.datastructures import Mesh
import math
import numpy as np

In [3]:
viewer = Viewer()
viewer.scene.clear()

參數設定

In [2]:
R = 3.0                       # 螺旋半徑
pitch = 40.0                  # 每一圈上升高度（節距）
turns = 0.25                  # 總圈數
points_per_turn = 200         # 每圈取樣點（越大越平滑）
rungs_per_turn = 100          # 每圈樓板數
RUNG_WIDTH = 3.0              # 樓板寬度
flor = pitch / rungs_per_turn # 樓層高度 (dz)

WIND_DIR = Vector(1, 0, 0)    # 主風向（+X）
U = 1.0                       # 來流風速
R_OBS = R * 1.15              # DNＡ 視為障礙物的等效半徑
ALPHA = 1.8                   # 繞流強度

# 在 YZ 平面上佈點的範圍與密度
N_Y, N_Z = 2, 3
Y_RANGE = (-2.2 * R, 2.2 * R)
Z_RANGE = (0.0, pitch * turns)

# 流線參數
STREAM_LEN = 6.0 * R          # 每條流線的目標長度
STEP_SIZE  = 0.08 * R         # RK2 積分步長
SEG_LEN    = 0.20 * R         # 每段線段長度
GAP_LEN    = 0.08 * R         # 段與段之間的間隙
HEAD_SCALE = 0.25             # 箭頭長度比例

COLOR_LINE = Color(0.2, 0.6, 1.0, 0.9)   # 預設線色
COLOR_HEAD = Color(0.2, 0.6, 1.0, 0.9)   # 預設箭頭色

# 顏色映射：慢 → 快
COLOR_SLOW = Color(0.2, 0.6, 1.0, 0.9)
COLOR_FAST = Color(1.0, 0.3, 0.2, 0.9)

X_START = -5.0 * R            # 流線起點 X 座標（模型左側）

畫出DNA雙股

In [5]:
def helix_points(R, pitch, turns, points_per_turn, phase=0.0):
    """回傳位於 helix 上的一串 Point。"""
    pts = []
    total_steps = int(points_per_turn * turns) + 1
    for i in range(total_steps):
        t = 2.0 * math.pi * (i / points_per_turn + phase / (2.0 * math.pi))
        z = pitch * i / points_per_turn
        pts.append(Point(R * math.cos(t), R * math.sin(t), z))
    return pts

pts1 = helix_points(R, pitch, turns, points_per_turn, phase=0.0)
pts2 = helix_points(R, pitch, turns, points_per_turn, phase=math.pi)

poly1 = Polyline(pts1)
poly2 = Polyline(pts2)
viewer.scene.add(poly1, name="strand_A", color=Color(0.15, 0.45, 1.0))
viewer.scene.add(poly2, name="strand_B", color=Color(1.0, 0.25, 0.25))

# 建立樓板
def quad_at_center_and_dir(center: Point, u_dir: Vector, length: float, width: float):
    u = u_dir.copy()
    if u.length == 0:
        u = Vector(1, 0, 0)
    u.unitize()

    v = Vector(0, 0, 1).cross(u)
    if v.length == 0:
        v = Vector(1, 0, 0).cross(u)
    v.unitize()

    halfL, halfW = 0.5 * length, 0.5 * width
    a = center + (u * halfL) + (v * halfW)
    b = center - (u * halfL) + (v * halfW)
    c = center - (u * halfL) - (v * halfW)
    d = center + (u * halfL) - (v * halfW)
    return [a, b, c, d]

def prism_from_quad_and_height(quad, height):
    lift = Vector(0, 0, height)
    a, b, c, d = quad
    a2, b2, c2, d2 = a + lift, b + lift, c + lift, d + lift
    vertices = [a, b, c, d, a2, b2, c2, d2]
    faces = [
        [0, 1, 2, 3],  # bottom
        [4, 5, 6, 7],  # top
        [0, 1, 5, 4],
        [1, 2, 6, 5],
        [2, 3, 7, 6],
        [3, 0, 4, 7],
    ]
    return Mesh.from_vertices_and_faces(vertices, faces)

extrude_height = flor
N = int(turns * rungs_per_turn)

for n in range(N):
    t = 2.0 * math.pi * (n / rungs_per_turn)
    zc = n * flor

    p1 = Point(R * math.cos(t),  R * math.sin(t),  zc)
    p2 = Point(-R * math.cos(t), -R * math.sin(t), zc)
    center = Point(0, 0, zc)

    u_dir = Vector.from_start_end(p1, p2)
    length = (p1 - p2).length
    quad = quad_at_center_and_dir(center, u_dir, length, RUNG_WIDTH)
    solid = prism_from_quad_and_height(quad, extrude_height)

    col = Color(0.80, 0.80, 0.85) if (n % 2 == 0) else Color(0.62, 0.62, 0.68)
    viewer.scene.add(solid, color=col)

CFD模擬

In [ ]:
segments = []       #save (p0,p1,speed)
wind_vectors = []   #save wind vector at each segment

In [7]:
def swirl_around_DNA(p: Point, R: float, alpha: float, R_obs: float) -> Vector:
    """
    產生繞著 DNA 中軸（沿 X 方向）的旋流速度分量。
    - 強度在靠近柱體時比較大，往外逐漸衰減
    - 方向沿著 YZ 平面的切線
    """

    # 這裡假設 DNA 中心在 Y/Z 範圍的中線（要跟你 velocity_field 用的一樣）
    cy = 0.5 * (Y_RANGE[0] + Y_RANGE[1])
    cz = 0.5 * (Z_RANGE[0] + Z_RANGE[1])

    dy = p.y - cy
    dz = p.z - cz
    r2 = dy * dy + dz * dz

    # 在軸心上就不旋轉，避免除以 0
    if r2 < 1e-12:
        return Vector(0.0, 0.0, 0.0)

    r = math.sqrt(r2)

    # YZ 平面上的切線方向（順時針 / 逆時針都可以，這裡是 -sin / cos）
    t_y = -dz / r
    t_z =  dy / r

    # 旋流強度：距離越遠越小，在 R_obs 外快速衰減
    base = alpha * (R_obs ** 2 / max(r2, R_obs ** 2))

    # 在實體柱體附近可稍微加強一點
    if r < R:
        base *= 1.5

    # 只在 YZ 平面產生旋流，所以 X 分量為 0
    return Vector(0.0, base * t_y, base * t_z)

def velocity_field(p: Point) -> Vector:
    # 基本風
    v = WIND_DIR.unitized() * U

    # DNA 中心線
    cy = 0.5 * (Y_RANGE[0] + Y_RANGE[1])
    cz = 0.5 * (Z_RANGE[0] + Z_RANGE[1])
    dy, dz = p.y - cy, p.z - cz
    r = math.sqrt(dy*dy + dz*dz)

    # 旋流
    swirl = swirl_around_DNA(p, R, ALPHA, R_OBS)
    v = v + swirl

    # ❶ 加入擋風減速（越靠近 DNA → 越慢）
    if r < R_OBS:
        slow = (r / R_OBS)**2     # 0~1
        v = v * slow              # 減速模型

    # ❷ DNA 後方加入尾流（wake）
    if p.x > 0:
        wake = math.exp(-(r / (1.5 * R)))
        v = v * wake

    # ❸ 不要 unitize!! （這會破壞速度資訊）
    # return v.unitized()
    return v

def rk2(p: Point, h: float) -> Point:
    """2 階 Runge–Kutta 積分一步（較平滑）。"""
    k1 = velocity_field(p)
    mid = p + k1 * (0.5 * h)
    k2 = velocity_field(mid)
    return p + k2 * h

MAX_OBJECTS = 8000
_obj_count = 0

def draw_arrow(p0: Point, p1: Point, col: Color):
    """實際在 viewer 上畫線段＋箭頭。"""
    global _obj_count
    if _obj_count >= MAX_OBJECTS:
        return

    # 主線段
    viewer.scene.add(Line(p0, p1), color=col)
    _obj_count += 1

    # 箭頭（末端的小 V）
    dirv = Vector.from_start_end(p0, p1)
    L = dirv.length
    if L < 1e-9:
        return
    u = dirv.unitized()

    # 找一個垂直方向
    v = Vector(0, 0, 1).cross(u)
    if v.length < 1e-9:
        v = Vector(0, 1, 0).cross(u)
    v.unitize()

    head_len = HEAD_SCALE * L
    head_wid = 0.5 * HEAD_SCALE * L
    tip  = p1
    base = p1 - u * head_len
    left = base + v * head_wid
    right = base - v * head_wid

    viewer.scene.add(Line(left, tip),  color=col)
    viewer.scene.add(Line(right, tip), color=col)
    _obj_count += 2

def add_streamline(seed: Point):
    """從 seed 出發積分一條流線，並將線段存入 segments。"""
    total = 0.0
    cur = seed
    carry = 0.0
    seg_start = cur

    zc_mid = 0.5 * (Z_RANGE[0] + Z_RANGE[1])

    while total < STREAM_LEN:
        nxt = rk2(cur, STEP_SIZE)
        step_vec = Vector.from_start_end(cur, nxt)
        step_d = step_vec.length

        # 風速（用目前位置的速度）
        v = velocity_field(cur)
        speed = v.length
        wind_vectors.append(v)

        # 避免穿過 建築：如果進入保護圈，就沿切線推開一點
        if (cur.y ** 2 + (cur.z - zc_mid) ** 2) < (0.9 * R_OBS) ** 2:
            phi = math.atan2(cur.z - zc_mid, cur.y)
            push = Vector(-math.sin(phi), math.cos(phi), 0.0) * (0.05 * R)
            cur = cur + push
            continue

        carry += step_d
        total += step_d
        cur = nxt

        if carry >= SEG_LEN:
            segments.append((seg_start, cur, speed))
            dirv = Vector.from_start_end(seg_start, cur)
            if dirv.length > 1e-9:
                diru = dirv.unitized()
                seg_start = cur + diru * GAP_LEN
                cur = seg_start
            carry = 0.0

def add_windfield():
    """在整個 YZ 範圍上佈置多條流線。"""
    y0, y1 = Y_RANGE
    z0, z1 = Z_RANGE
    for iy in range(N_Y):
        y = y0 + (y1 - y0) * (iy + 0.5) / N_Y
        for iz in range(N_Z):
            z = z0 + (z1 - z0) * (iz + 0.5) / N_Z
            seed = Point(X_START, y, z)
            add_streamline(seed)

def compute_wind_score(wind_vectors, U_inf: float, rho: float = 1.2):
    """根據速度場給出簡單的平均風速與『動量虧損』評分。"""
    if not wind_vectors:
        return {"avg_speed": 0.0, "momentum_deficit": 0.0, "score": 0.0}

    speeds = np.array([v.length for v in wind_vectors], dtype=float)

    speeds_clipped = np.minimum(speeds, U_inf)
    deficit = rho * (U_inf ** 2 - speeds_clipped ** 2)
    DeltaM = float(np.sum(deficit))

    worst = rho * (U_inf ** 2) * len(speeds_clipped)
    score = (worst - DeltaM) / worst * 100.0
    score = float(np.clip(score, 0.0, 100.0))

    return {
        "avg_speed": float(np.mean(speeds)),
        "momentum_deficit": DeltaM,
        "score": score,
    }

風場

In [8]:
add_windfield()

# 根據速度，把 segments 上色並畫出箭頭
if segments:
    speeds = [s for _, _, s in segments]
    s_min, s_max = min(speeds), max(speeds)
    if abs(s_max - s_min) < 1e-8:
        s_max = s_min + 1.0

    def lerp_color(c0, c1, t):
        return Color(
            c0.r + (c1.r - c0.r) * t,
            c0.g + (c1.g - c0.g) * t,
            c0.b + (c1.b - c0.b) * t,
            c0.a + (c1.a - c0.a) * t,
        )

    for p0, p1, s in segments:
        t = (s - s_min) / (s_max - s_min)
        col = lerp_color(COLOR_SLOW, COLOR_FAST, t)
        draw_arrow(p0, p1, col)

計算評分

In [9]:
result = compute_wind_score(wind_vectors, U_inf=U)

print("📊 CFD 風場評分結果：")
print(f"  平均風速           : {result['avg_speed']:.3f}")
print(f"  動量虧損 (指標)   : {result['momentum_deficit']:.3f}")
print(f"  評分 (0–100, 越高越好): {result['score']:.1f}")

📊 CFD 風場評分結果：
  平均風速           : 1.205
  動量虧損 (指標)   : 108.014
  評分 (0–100, 越高越好): 76.1


In [10]:
viewer.show()